In [1]:
import ee
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import geemap
import os
from sklearn.cluster import KMeans, MiniBatchKMeans, DBSCAN, MeanShift, OPTICS, SpectralClustering
from sklearn.mixture import GaussianMixture
from osgeo import gdal
import numpy as np
import json
from matplotlib.ticker import FuncFormatter
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import BernoulliRBM
from sklearn.preprocessing import normalize, StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from tqdm import tqdm
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

ee.Initialize()

# json_file = 'GEE_LULC_area.txt'
# with open(json_file) as json_file:
#     geoJSON = json.load(json_file)

lulc_classes = {10: 'Trees',\
20: 'Shrubland',\
30: 'Grassland',\
40: 'Cropland',\
50: 'Built-up',\
60: 'Barren / sparse vegetation',\
70: 'Snow and ice',\
80: 'Open water',\
90: 'Herbaceous wetland',\
95: 'Mangroves',\
100: 'Moss and lichen'}

In [2]:
def generate_rect(point, rect_size = 0.1):
    aoi_area = ee.Geometry.Rectangle(point[0], point[1], point[0] + rect_size, point[1] + rect_size)
    return aoi_area

def geoJSON2eeRect(geoJSON):
    coords = geoJSON['features'][0]['geometry']['coordinates']
    area = ee.Geometry.Polygon(coords)
    return area

In [3]:
def NDVI(img):
    ndvi = ee.Image(img.normalizedDifference(['B8', 'B4'])).rename(["NDVI"])
    return ndvi

def NDWI(img):
    ndwi = ee.Image(img.normalizedDifference(['B3', 'B8'])).rename(["NDWI"])
    return ndwi

def NDII(img):
    ndii = ee.Image(img.normalizedDifference(['B8', 'B11'])).rename(["NDII"])
    return ndii

def sNDVI(img):
    sndvi = ee.Image(img.normalizedDifference(['B3', 'B4'])).rename(["sNDVI"])
    return sndvi

def VDVI(img):
    vdvi = img.expression(
        '((2 * GREEN) - RED - BLUE) / ( (2 * GREEN) + RED + BLUE)', {
          'GREEN': img.select('B3'),
          'RED': img.select('B4'),
          'BLUE': img.select('B2')}).rename(["VDVI"])
    return vdvi

def TDVI(img):
    tdvi = img.expression(
        '1.5 * (NIR - RED) / sqrt(NIR ** 2 + RED + 0.5)', {
          'NIR': img.select('B8'),
          'RED': img.select('B4')}).rename(["TDVI"])
    return tdvi

def ExGI(img):
    exgi = img.expression(
        '2 * GREEN - (RED + BLUE)', {
          'GREEN': img.select('B3'),
          'RED': img.select('B4'),
          'BLUE': img.select('B2')}).rename(["ExGI"])
    return exgi

def GCC(img):
    gcc = img.expression(
        'GREEN / (RED + GREEN + BLUE)', {
          'GREEN': img.select('B3'),
          'RED': img.select('B4'),
          'BLUE': img.select('B2')}).rename(["GCC"])
    return gcc

def EVI(img):
    evi = img.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
          'NIR': img.select('B8'),
          'RED': img.select('B4'),
          'BLUE': img.select('B2')}).rename(["EVI"])
    return evi

def EVI2(img):
    evi2 = img.expression(
        '2.5 * ((NIR - RED) / (NIR + 2.4 * RED + 1))', {
          'NIR': img.select('B8'),
          'RED': img.select('B4')}).rename(["EVI2"])
    return evi2

def RVI(img):
    rvi = img.expression(
        'RED/NIR', {
          'NIR': img.select('B8'),
          'RED': img.select('B4')}).rename(["RVI"])
    return rvi

def ARVI(img):
    arvi = img.expression(
        '(NIR-RED * BLUE) / (NIR+RED * BLUE)', {
          'NIR': img.select('B8'),
          'RED': img.select('B4'),
          'BLUE': img.select('B2')}).rename(["ARVI"])
    return arvi

def GARI(img):
    gari = img.expression(
        '(NIR–(GREEN – (1.70 * (BLUE – RED)))) / (NIR + (GREEN – (1.70 * (BLUE – RED))))', {
          'NIR': img.select('B8'),
          'RED': img.select('B4'),
          'GREEN': img.select('B3'),
          'BLUE': img.select('B2')}).rename(["GARI"])
    return gari

def VARI(img):
    vari = img.expression(
        '(GREEN - RED) / (GREEN + RED - BLUE)', {
          'RED': img.select('B4'),
          'GREEN': img.select('B3'),
          'BLUE': img.select('B2')}).rename(["VARI"])
    return vari

def DVI(img):
    dvi = img.expression(
        'NIR - RED', {
          'NIR': img.select('B8'),
          'RED': img.select('B4')}).rename(["DVI"])
    return dvi

def MSAVI2(img):
    msavi2 = img.expression(
        '(2 * NIR + 1 - (((2 * NIR + 1) ** 2 - 8 * (NIR - RED)) ** 0.5)) / 2', {
          'NIR': img.select('B8'),
          'RED': img.select('B4')}).rename(["MSAVI2"])
    return msavi2

def SAVI(img):
    savi = img.expression(
        '((NIR - RED) / (NIR + RED + 0.5)) * 1.5', {
          'NIR': img.select('B8'),
          'RED': img.select('B4')}).rename(["SAVI"])
    return savi

def LAI(img):
    lai = img.expression(
        '-log10((0.69 - ((NIR - RED) / (NIR + RED + 0.5)) * 1.5) / 0.59) / 0.91', {
          'NIR': img.select('B8'),
          'RED': img.select('B4')}).rename(["LAI"])
    return lai

def MCARI(img):
    mcari = img.expression(
        '(REDEDGE - RED) - 0.2 * (REDEDGE - GREEN) * (REDEDGE / RED)', {
          'RED': img.select('B4'),
          'GREEN': img.select('B3'),
          'REDEDGE': img.select('B5')}).rename(["MCARI"])
    return mcari

def SIPI(img):
    sipi = img.expression(
        '(B8 - B1) / (B8 - B4) ', {
          'B1': img.select('B1'),
          'B4': img.select('B4'),
          'B8': img.select('B8')}).rename(["SIPI"])
    return sipi

def GLI(img):
    gli = img.expression(
        '((GREEN - RED) + (GREEN - BLUE)) / ((2 * GREEN) + (BLUE + RED))', {
          'RED': img.select('B4'),
          'GREEN': img.select('B3'),
          'BLUE': img.select('B2')}).rename(["GLI"])
    return gli

def LCI(img):
    lci = img.expression(
        '(NIR - REDEDGE) / (NIR + RED)', {
          'RED': img.select('B4'),
          'NIR': img.select('B8'),
          'REDEDGE': img.select('B5')}).rename(["LCI"])
    return lci

def OSAVI(img):
    osavi = img.expression(
        '((NIR - RED) / (NIR + RED + 0.16))', {
          'NIR': img.select('B8'),
          'RED': img.select('B4')}).rename(["OSAVI"])
    return osavi


def SATVI(img):
    satvi = img.expression(
        '((SWIR1 - RED) / (SWIR1 + RED + 0.5) * (1 + 0.5) - SWIR2 / 2)', {
          'RED': img.select('B4'),
          'SWIR1': img.select('B11'),
          'SWIR2': img.select('B12')}).rename(["SATVI"])
    return satvi

def AVI(img):
    avi = img.expression(
        '((B4 + 1) * (256 - B3) * (B4 - B3)) ** (1 / 3)', {
          'B3': img.select('B3'),
          'B4': img.select('B4')}).rename(["AVI"])
    return avi

def BI(img):
    bi = img.expression(
        '((B4 + B2) - B3) / ((B4 + B2) + B3)', {
          'B3': img.select('B3'),
          'B4': img.select('B4'),
          'B2': img.select('B2')}).rename(["BI"])
    return bi

def SI(img):
    si = img.expression(
        'sqrt((256 - B2) * (256 - B3))', {
          'B2': img.select('B2'),
          'B3': img.select('B3')}).rename(["SI"])
    return si

def B8(img):
    return img.select('B8')


def img_(img):
    return img

In [34]:
def LatLonImg(img):
    img = img.addBands(ee.Image.pixelLonLat())
    img = img.reduceRegion(reducer=ee.Reducer.toList(),\
                                        geometry=area,\
                                        maxPixels=1e13,\
                                        scale=10);
    data = np.array((ee.Array(img.get("result")).getInfo()))
    lats = np.array((ee.Array(img.get("latitude")).getInfo()))
    lons = np.array((ee.Array(img.get("longitude")).getInfo()))
    return lats, lons, data

def toImage(lats,lons,data):
    uniqueLats = np.unique(lats)
    uniqueLons = np.unique(lons)
    ncols = len(uniqueLons)
    nrows = len(uniqueLats)
    ys = uniqueLats[1] - uniqueLats[0]
    xs = uniqueLons[1] - uniqueLons[0]
    arr = np.zeros([nrows, ncols], np.float32) #-9999
    counter =0
    for y in range(0,len(arr),1):
        for x in range(0,len(arr[0]),1):
            if lats[counter] == uniqueLats[y] and lons[counter] == uniqueLons[x] and counter < len(lats)-1:
                counter+=1
                arr[len(uniqueLats)-1-y,x] = data[counter] # we start from lower left corner
    return arr

def format_xtick_labels(x, pos): return '{0:.3f}$^\circ$N'.format(x)
def format_ytick_labels(x, pos): return '{0:.3f}$^\circ$E'.format(x)

def gee2Array(collection, area, date_start, date_end, ind, suppress_plot = True):
    acq_times = collection.aggregate_array('system:time_start').getInfo()
    im_list = collection.toList(collection.size())
    dates = [time.strftime('%x', time.gmtime(acq_time/1000)) for acq_time in acq_times]
    myCollection = collection.map(ind)
    result = ee.Image(myCollection.median()).rename(['result'])
    lat, lon, data = LatLonImg(result)
    image  = toImage(lat, lon, data)
    
    rows = np.shape(image)[0]
    cols = np.shape(image)[1]

    data = image.ravel()
    for i in range(1, len(data) + 1):
        data[:] = data.flatten()
    data = data.reshape(-1, 1)
    if not suppress_plot:
        band_name = myCollection.getInfo()['features'][0]['bands'][0]['id']
        title1 = collection.getInfo()['id']
        cmap = 'viridis'

        fig = plt.figure(figsize = (20, 20))
        ax = fig.add_subplot(2, 2, 1)
        plot1 = ax.imshow(image, extent = (lat.min(), lat.max(), lon.min(), lon.max()), cmap = cmap)
        plt.gca().xaxis.set_major_formatter(FuncFormatter(format_xtick_labels))
        plt.gca().yaxis.set_major_formatter(FuncFormatter(format_ytick_labels))
        plt.grid(ls = ":", zorder = 100)
        ax.set_title('Visualization of band {} from \nsatellite {}'.format(band_name, title1), fontsize = 17)
        fig.colorbar(plot1)
    return(lat, lon, image)

def cluster(X, y):
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size= 0.30)
    y_train = np.array(y_train)
    y_test = np.array(y_test)

    myscaler = StandardScaler()
    myscaler.fit(x_train)
    x_train = myscaler.transform(x_train)
    x_test = myscaler.transform(x_test)

    # clf = MLPClassifier(hidden_layer_sizes=(100,100,100), max_iter=500)
    clf = MLPClassifier(hidden_layer_sizes=(9, 12, 10, 15), activation='relu', solver='adam', max_iter=2000)

    clf.fit(x_train, y_train.ravel())
    y_pred = clf.predict(x_test)

    print('The accuracy score of the classification is {:.1%}'.format(accuracy_score(y_test, y_pred)))
    print('F1-Score is {:.1%}'.format(f1_score(y_test, y_pred, average = 'micro')))
    return(y_pred)

In [68]:
date_start = '2018-06-15'
date_stop = '2018-07-25'

cor_x, cor_y = 40.572292, 21.684652
# area = geoJSON2eeRect(geoJSON)

area = generate_rect([cor_y, cor_x], rect_size = 0.03)

image_lan = (ee.ImageCollection("LANDSAT/LC08/C01/T1")\
                .filterBounds(area)\
                .filterDate(ee.Date(date_start),ee.Date(date_stop))\
                .sort('system:time_start'))

image_sen = (ee.ImageCollection("COPERNICUS/S2_SR")\
                .filterBounds(area)\
                .filterDate(ee.Date(date_start),ee.Date(date_stop))\
                .sort('system:time_start'))

image_lulc = ee.ImageCollection("ESA/WorldCover/v100").select(['Map'])\
                .filterBounds(area)

In [69]:
lat, lon, lan = gee2Array(image_lan, area, date_start, date_stop, B8)

In [ ]:
lat, lon, sen = gee2Array(image_sen, area, date_start, date_stop, B8)

In [ ]:
lat, lon, mod = gee2Array(image_lulc, area, date_start, date_stop, img_)

In [ ]:
inds = [NDVI, NDWI, NDII, sNDVI, VDVI, TDVI, ExGI, GCC, EVI, EVI2, RVI, VARI, DVI, MSAVI2, SAVI, LAI, MCARI, SIPI, GLI, LCI, OSAVI, SATVI, BI, SI]

train_sen_x = []
for i in tqdm(inds):
    train_sen_x.append(gee2Array(image_sen, area, date_start, date_stop, i)[-1].ravel())
train_sen_x = pd.DataFrame(train_sen_x).T

lulc_y = gee2Array(image_lulc, area, date_start, date_stop, img_, suppress_plot = False)[-1].ravel()
lulc_y = pd.DataFrame(lulc_y)

In [ ]:
pred_all = cluster(train_sen_x, lulc_y)
pd.DataFrame(pred_all).replace(lulc_classes).value_counts().sort_index()

In [ ]:
pred_two = cluster(pd.DataFrame([sen.ravel(), lan.ravel()]).T, lulc_y)
pd.DataFrame(pred_two).replace(lulc_classes).value_counts().sort_index()

In [ ]:
lulc_y.replace(lulc_classes).value_counts().sort_index()